Name: Deniz Ekiz
ID: 2016700051
## Homework 3 Programming Part

### Question
In this exercise we model a string of text using a Markov(1) model. For simplicity we only consider
letters ’a-z’. Capital letters ’A-Z’ are mapped to the corresponding ones. All remaining letters,
symbols, numbers, including spaces, are denoted by ’.’.
We have a probability table T where Ti,j = p(xt = j|xt−1 = i) transition model of letters in
English text for t = 1, 2 . . . N. Assume that the initial letter in a string is always a space denoted
as x0 = ’.’. Such a model where the probability table is always the same is sometimes called a
stationary model.


4. Discuss how you can improve the model to get better estimations



In [1]:
test_strings = ['th__br__n.f_x.', '_u_st__n_.to_be._nsw_r__','i__at_._a_h_n_._e_r_i_g','q___t.___z._____t.__.___.__.']

Hint: The code below loads a table of transition probabilities for English text.

In [2]:
import csv
import numpy as np
from IPython.display import display, Latex

alphabet = [chr(i+ord('a')) for i in range(26)]
alphabet.append('.')
letter2idx = {c:i for i,c in enumerate(alphabet)}

T = []
with open('transitions.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        T.append(row)

print('Example')
## p(x_t = 'u' | x_{t-1} = 'q')
display(Latex(r"$p(x_t = \text{'u'} | x_{t-1} = \text{'q'})$"))
print(T[letter2idx['q']][letter2idx['u']])
display(Latex(r"$p(x_t | x_{t-1} = \text{'a'})$"))
t = 0.0
for c,p in zip(alphabet,T[letter2idx['a']]):
    print(c,p)

Example


<IPython.core.display.Latex object>

0.9949749


<IPython.core.display.Latex object>

('a', '0.0002835')
('b', '0.0228302')
('c', '0.0369041')
('d', '0.0426290')
('e', '0.0012216')
('f', '0.0075739')
('g', '0.0171385')
('h', '0.0014659')
('i', '0.0372661')
('j', '0.0002353')
('k', '0.0110124')
('l', '0.0778259')
('m', '0.0260757')
('n', '0.2145354')
('o', '0.0005459')
('p', '0.0195213')
('q', '0.0001749')
('r', '0.1104770')
('s', '0.0934290')
('t', '0.1317960')
('u', '0.0098029')
('v', '0.0306574')
('w', '0.0088799')
('x', '0.0009562')
('y', '0.0233701')
('z', '0.0018701')
('.', '0.0715219')


# Part 1
 For a given N, write a program to sample random strings with letters x1, x2, . . . , xN from p(x1:N|x0)

For N = 1 it is trivial. We will just need to select the corresponding section of conditional probability table.  Then make a random choice using np.random.choice function.

$p(x_1|x_0 = \text{'.'})$

In [3]:
N = 1
#convert the string matrix to float. This is one time operation. Otherwise, we cant make calculations.
T = [[float(y) for y in x] for x in T]
# Assign first column of conditional probability table to a variable called guess_table
#letter2idx is key-value pair. It stores the corresponding value for each letter.
#Ex: letter2idx['a'] -> 0
#  letter2idx['b'] -> 1 , letter2idx['.'] -> 26
guess_table = T[letter2idx['.']]
print(np.random.choice(alphabet,1,p=guess_table))

['t']


Now let's look where N = 2 to illustrate this.

$p(x_{1:2} | x_0=\text{'.'})=p(x_1,x_2|x_0=\text{'.'})=\sum_{x_1,x_2}p(x_1|x_0 = \text{'.'})p(x_2|x_1)$

In [4]:
N = 2
#T[i][j] means p(x_{j}|n_{i})
guess_table = np.zeros(shape=(27*27,1))
guess_table_margin = np.zeros(shape=(27,1))
for i in range(0,len(alphabet)):
    for j in range(0,len(alphabet)):
        #print("x_0= = '.' " +"x1="+alphabet[i]+" x2="+alphabet[j]+" "+ str(np.dot(T[letter2idx['.']][i],T[i][j])))
        guess_table[i*27+j] = np.dot(T[i][j],T[letter2idx['.']][i],)
        guess_table_margin[i][0] += np.dot(T[i][j],T[letter2idx['.']][i],) 
#normalize
guess_table = np.divide(guess_table,np.sum(guess_table))

Let's make a choice from this array.

In [5]:
number = np.random.choice(range(0,27*27),p = guess_table[:,0])
x_1 = np.mod(number,27)
x_2 = np.ceil(number/27)
print(str(alphabet[x_1]),str(alphabet[int(x_2)]))

('.', 's')


We computed random string generation using markov model for N=1 and N=2. However, this array structure is not suitable for bigger N. I did it in this way in the beginning because, I am trying to learn.

In the end our output is a table, where the dimensions are 27xN. for x1,x2...xN . We make our selections from that table N times. Let's re formulaze it. In  order achive it we need to marginalize x1, x2 .. Xn

In order to check if we approach in correct way. Marginalize over x2 p(x1,x2|x0) to p(x1|xo). Let's see if they are equal.

In [6]:


#print(np.divide(guess_table_margin,np.sum(guess_table_margin)))
print(T[letter2idx['.']])

another_table = np.zeros((27,1))
first_table = T[letter2idx['.']]



[0.1062437, 0.0444502, 0.03916, 0.0282947, 0.0213084, 0.0400793, 0.0171783, 0.0606047, 0.0678165, 0.003466, 0.0045451, 0.0243019, 0.0406429, 0.0234882, 0.064992, 0.0273498, 0.0022208, 0.0214068, 0.0704687, 0.1460781, 0.0092399, 0.0079497, 0.0606385, 0.0001107, 0.0114638, 0.0002911, 0.0562102]


Hopefully, they are equal.

We need to find what is $p(x_{1:N} | x_0)$

$p(x_{1:N} | x_0=\text{'.'})=\sum_{x_1,x_2,....,X_N}p(x_1|x_0 = \text{'.'})p(x_2|x_1)p(x_3|x_2)...p(x_{N}|x_{N-1}) $

Given x_0 = '.', 27X27 conditional probability table T and number of letters N.

Output, guess_table -> 27xN probability table which contains every probability of states x1 x2, ... xn with respect to letters

Algorithm should be, 

i = 0

Start with p(x1|x_0 ='.'), which is row vector.  The values are the first column of the guess_table
p(x2|x1) is my probability table T. 
i++

multiply every x1 and x2. Marginalize x1.  The values are the second column of guess_table
i++
multiple every x3 in T and x2 in guess_table. Marginalize on x2 the values are the third column of guess table.
i ++

return guess_table

In [7]:
#27xN shaped matrix
# given j  z
N = 1000
guess_table = np.zeros((27,N+3))
x_0 = '.'
x_prev = x_0
#fill guess_table
word = np.zeros((0,1))
for i in range(0,len(alphabet)):
    guess_table[i,0] = T[letter2idx[x_prev]][i]
guess_table[:,0] = np.divide(guess_table[:,0],np.sum(guess_table[:,0]))
#print(guess_table[:,0])
word = np.vstack((word,str(np.random.choice(alphabet,1,p = guess_table[:,0])[0])))

for i in range(1,N):
    for j in range(0,27):
        for z in range(0,27):
            #print("x1="+alphabet[z]+" x2="+alphabet[j]+" "+ str(np.dot(T[z][j],guess_table[z,i-1])))
            #compute and marginalize
            guess_table[j,i] += np.dot(T[z][j],guess_table[z,i-1])
    #normalize
    guess_table[:,i] = np.divide(guess_table[:,i],np.sum(guess_table[:,i]))
    #make choice
    word = np.vstack((word ,str(np.random.choice(alphabet,1,p = guess_table[:,i])[0])))  
for i in range(0,len(word)):
    print(str(word[i,0])),
#guess_table = np.divide(guess_table,np.sum(guess_table))
#guess_table[0,:]

u n r s e d d o a n . h . z . l w e e d a b . . . i . h a l e . s s c b f t . e e a r . r . e . d r e o t a t a . h f a b b g r d g . p g d m t . i . . i l i h a . . . a l g t e o o h m e t i e . u t . b . r e o h a t a t h . . i e g e k . t n r k c r e n t t i n o e . . f f r y e s s . . i g t . v t . . . d z a i a e r . o o r n a i i a l a d . t . r . e f o i . v . e r t r . h l b f t i a r r h t . s h l a n . n g e n b . e e . . o n . . n r m t g d f i t t r f . u o m c d s e i h h o t b o . p . o r y s c e a r i o r s . e h u i d t t t e p a a h e u y u h l s a d a r r e l f s . s f u l . a o d h t s t s t e e l r r i . . t r e m . . . r . i e i s k g y . . r . . o a o o . . . n c t v e r e e a h e c y . t d t e n a c s i . . e x . m . e n i o h c e o . e l . m t i e n . r a t e a b n . e . r c . w d h o e y . r o . h s s t e h s r w t r d w e . v u h u o u m b n t c g i l f e n e a h t r . . h c e e t . n . . . t a c a n w . u s d s r i . e o a i s l i . d s . o t . l . j . a e l 

We can also write the second element of Probability (guess) table as 

$T^{T}T[letter2idx['.']$

In [8]:
print "The distribution of x2 , product of two distributions computed, then marginalization applied"
print("x2="),
print(guess_table[:,1])
print "The distribution of x2 , product of transpoze of the transition table and p(x1|x0='.') computed "
print("x2="),
print(np.dot(np.transpose(T),T[letter2idx['.']]))
print "The result is same, no normalization is applied, however, I will always apply normalization "

The distribution of x2 , product of two distributions computed, then marginalization applied
x2= [ 0.06595029  0.00781004  0.01517336  0.01913779  0.13188224  0.01555645
  0.00930034  0.07811046  0.06348793  0.00062664  0.00532751  0.03583539
  0.01471353  0.05857413  0.07152386  0.01149852  0.00040637  0.04863329
  0.04157748  0.05900106  0.03189465  0.00719343  0.00954437  0.000513
  0.01585309  0.00067948  0.18019532]
The distribution of x2 , product of transpoze of the transition table and p(x1|x0='.') computed 
x2= [ 0.06595029  0.00781004  0.01517336  0.01913779  0.13188224  0.01555645
  0.00930034  0.07811046  0.06348793  0.00062664  0.00532751  0.03583539
  0.01471353  0.05857413  0.07152386  0.01149852  0.00040637  0.04863329
  0.04157748  0.05900106  0.03189465  0.00719343  0.00954437  0.000513
  0.01585309  0.00067948  0.18019532]
The result is same, no normalization is applied, however, I will always apply normalization 


## Part 2
Question: Now suppose you are given strings with missing letters, where each missing letter is denoted
by a question mark (or underscore, as below). Implement a method, that samples missing
letters conditioned on observed ones, i.e., samples from p(x−α|xα) where α denotes indices
of observed letters. For example, if the input is ’t??.’, we have N = 4 and x1 = ’t’ and
x4 = ’.’, α = {1, 4} and −α = {2, 3}. Your program may possibly generate the strings ’the.’,
’twi.’, ’tee.’, etc. Hint: make sure to make use all data given and sample from the correct
distribution. Implement the method and print the results for the test strings below.

We have sucessfully generated our forward markov(1) model. Now let's turn it into a function

In [9]:
def straight_markov(first_char,N,T,repeat,start,p,l):
    #27xN shaped matrix
    # given j  z
    guess_table = np.zeros((27,N+2))
    x_0 = first_char
    x_prev = x_0
    #fill guess_table
    word = np.zeros((0,1))
    for i in range(0,len(alphabet)):
        guess_table[i,0] = T[letter2idx[x_prev]][i]
    guess_table[:,0] = np.divide(guess_table[:,0],np.sum(guess_table[:,0]))
    #print(guess_table[:,0])
    for i in range(1,N):
        for j in range(0,27):
            for z in range(0,27):
                #print("x1="+alphabet[z]+" x2="+alphabet[j]+" "+ str(np.dot(T[z][j],guess_table[z,i-1])))
                #compute and marginalize
                guess_table[j,i] += np.dot(T[z][j],guess_table[z,i-1])
        #normalize
        guess_table[:,i] = np.divide(guess_table[:,i],np.sum(guess_table[:,i]))
    for i in range(0,repeat):
        word = np.zeros((0,N))
        word = np.vstack(first_char)
        for j in range(0,N):
            if not l:
                word = np.vstack((word ,str(np.random.choice(alphabet,1,p = guess_table[:,j])[0]))) 
            else:
                word = np.vstack((word ,str(alphabet[np.argmax(guess_table[:,j])])))
        if p :
            print(''.join(word[start:,0]))
        else:
            return word[start:]

Let's call our function with x_0 = 't' , N=4, 10 times

In [10]:
straight_markov('t',3,T,10,0,p=True,l=False)

ttey
t.ue
t...
tian
thde
thrc
teae
thmt
toab
tldh


The markov model is like x0 -> x1 -> x2-> .... xn -> xn+1

Let's look at for a _ _ b for getting some intuition. This model is p(x2,x3|x1=a,x4=b). We need to construct our table using these facts.

$p(x_2,x_3|x_1=\text{'a'}, x_4=b)$ $\alpha$ $p(x_4=\text{'b'},x_3,x_2,x_1=\text{'a'}) $

$=p(x_4=\text{'b'}|x_3,x_2,x_1=\text{'a'})p(x_3,x_2,x_1=\text{'a'}) $

$=p(x_4=\text{'b'}|x_3,x_2,x_1='a')p(x_3|x_2,x_1='a')p(x_2,x_1='a')$

$=p(x_4=\text{'b'}|x_3,x_2,x_1=\text{'a'})p(x_3|x_2,x_1=\text{'a'})p(x_2|x_1=\text{'a'})p(x_1=\text{'a'}|x_o = \text{'.'})$

There is also x0 = '.' in this model, since we know that x0 = '.' 

p(x0 = '.') = 1








Applying conditional independences the equation turns out to be,

$= p(x_4=\text{'b'}|x_3)p(x_3|x_2)p(x_2|x_1 = \text{'a'})p(x_1 = \text{'a'}|x_0 = \text{'.'})$

$p(x_1 = 'a'|x_0 ='.')$ is a scalar.

$p(x_2|x_1=\text{'a'})$ is vector.

$p(x_3|x_2)$ is our probability table T.

$p(x4=\text{'b'}|x3)$ is a vector.

The result should be a matrix of 27x2, where the row indexes are the elements of domain. Columns are the unkown variables.

We need to compute it in a automatic way

The inference from future is not as trivial as forward inference.

There are two types of string pattern one is a... case. 

The other is a..b case. There is no such other case only the number of unkowns are changing.

I will identify the types of the strings with a function

For general case we can prove it with the following:


$ p(x_{1:N}|,x_0=\hat{x_0})$

$\alpha$

$p(x_{N+1}=\hat{x}_{N+1}|x_N)p(x_n|x_{n-1})\dots p(x_1|x_0=\hat{x}_0)$

The $p(x_1|x_0=\hat{x}_0)$ term can be obtained easily via row selection.

In order to obtain other terms we need to multiply T by N times. 

Then we need to take the transpoze of T.





In [11]:
########### Inference from Future START ###########  
def inference_from_future(first_char,last_char,N,T,start,l):
    prob = 1
    guess_table = np.zeros((27,N+2))
    x_0 = first_char
    x_prev = x_0
    word = np.zeros((0,1))
    #print("We have " + str(N) + " unkown ")
    if(first_char == ""):
        first_char = '.'
    #print("x_1"+ " = " + first_char)
    #print("x_"+str(N+1)+ " = " + last_char)
    for i in range(0,N):
        matrix = T
        for x_d in alphabet:
            #print("x_"+str(i + 2)+" = " + x_d ),
            guess_table[letter2idx[x_d],i] = T[letter2idx[first_char]][letter2idx[x_d]]
        else:
            for x_d in alphabet:
                #print("x_"+str(i + 2)+" = " + x_d ),
                guess_table[letter2idx[x_d],i] = guess_table[letter2idx[x_d],i]
        #for j in range(0,N):
        if l:
            matrix=np.transpose(np.linalg.matrix_power(T,N-i)) # decrease the number of N since we pretend like we have found the letter
        else:
            matrix=np.transpose(np.linalg.matrix_power(T,N))
        guess_table[:,i] = matrix[letter2idx[last_char]]*guess_table[:,i]
        guess_table[:,i] = np.divide(guess_table[:,i],np.sum(guess_table[:,i]))
        if l:
            first_char = str(alphabet[np.argmax(guess_table[:,i])])
            prob = np.dot(prob,guess_table[letter2idx[first_char],i])
            # assign argmax as first char.
        repeat = 1
    for i in range(0,repeat):
        word = np.zeros((0,N))
        word = np.vstack(first_char)
        for j in range(0,N):
            if not l:
                word = np.vstack((word ,str(np.random.choice(alphabet,1,p = guess_table[:,j])[0]))) 
            if l:
                #print np.argmax(guess_table[:,j])
                word = np.vstack((word ,str(alphabet[np.argmax(guess_table[:,j])])))
        else:
            if not l:
                return word[start:]
            if l:
                return word[start:],prob
    
########### Inference from Future END ###########  
########### STRING OPERATION FUNCTIONS START ###########  
def underscore_finder(my_str):
    list_of_underscore_indices = []
    for i in range(0,len(my_str)):
        if(my_str[i] == "_"):
            list_of_underscore_indices.append(i)
    return list_of_underscore_indices
def group_problems(list_of_underscore_indices):
    problems = []
    for i in range(0,len(list_of_underscore_indices)):
        problems.append([])
    #print(problems)
    z = 0
    for i in range(0,len(list_of_underscore_indices)):
        problems[z].append(list_of_underscore_indices[i])
        if(i == len(list_of_underscore_indices)-1):
            break
        if(list_of_underscore_indices[i + 1] - list_of_underscore_indices[i] !=  1):
            z = z + 1
    #delete empty lists
    return [x for x in problems if x != []]
            
    
def problem_identify(my_str,list_of_underscore_indices,T,l):
    str_len = len(my_str)
    problems = group_problems(list_of_underscore_indices)
    #print(problems)
    p_num = len(problems)
    pb = 1
    #print("The amount of problem is : " + str(p_num))
    for i in range(0,p_num):
        p_type = 0
        #p_type = 1 if straight markov
        #p_type = 2 if inference from future is need
        #print("Working on problem no: "+ str(i))
        problem = problems[i]
        if(problem[0] == 0):
            prev = ""
        else:
            prev = my_str[problem[0]-1]
        if(problem[-1] == (len(my_str) -1) ):
            next_letter = ""
            p_type = 1
        else:
            next_letter = my_str[problem[-1]+1]
            p_type = 2
        #print "type = ", str(p_type),prev,len(problem)*"_",next_letter, " length ",len(problem)
        if not l:
            my_str = inference(p_type,my_str,problem,prev,next_letter,T,l)
        else:
            my_str,prob = inference(p_type,my_str,problem,prev,next_letter,T,l)
            pb = np.dot(prob,pb)
    if l:
        print "Log prob:", np.log(pb)
    return my_str
########### STRING OPERATION FUNCTIONS END ########### 

def inference(p_type,my_str,indices,prev_letter,next_letter,T,l):
    prob = 1
    if(my_str == ""):
        #print("Null string error")
        return ""
    if(p_type == 1):
        #Straight markov
        #print("Problem type = 1")
        if prev_letter == "":
            result = straight_markov(first_char=".",T=T,N=len(indices),repeat=1,start=1,p=False,l=l)
            #print("Straight! 1")
            #print(result)
            a = 0
            for i in indices:
                list1 = list(my_str)
                list1[i] = str("".join(result[a]))
                a = a+1
                my_str = "".join(list1)
            #print "Result is ", my_str
        else:
            result = straight_markov(first_char=prev_letter,T=T,N=len(indices),repeat=1,start=1,p=False,l=l)
            #print("Straight! 2")
            #print(result)
            a = 0
            for i in indices:
                list1 = list(my_str)
                list1[i] = str("".join(result[a]))
                a = a+1
                my_str = "".join(list1)
            #print "Result is ", my_str
    elif(p_type == 2):
        if l:
            result,pb = inference_from_future(first_char = prev_letter,last_char = next_letter,N=len(indices),T=T,start = 1,l=l)
            prob = np.dot(pb,prob)
        else:
            result = inference_from_future(first_char = prev_letter,last_char = next_letter,N=len(indices),T=T,start = 1,l=l)

        #print("From future!")
        #print(result)
        a = 0
        for i in indices:
            list1 = list(my_str)
            list1[i] = str("".join(result[a]))
            a = a+1
            my_str = "".join(list1)
        #print "Result is ", my_str
    if l:
        return my_str,prob
    return my_str


In [12]:
for i in range(0,len(test_strings)):
    #repeat samples
    print "samples for " ,test_strings[i]
    for j in range(0,10):
        print(problem_identify(test_strings[i],underscore_finder(test_strings[i]),T,False))
#group_problems(underscore_finder(test_strings[0]))
#print(test_strings[1])
#problem_identify(test_strings[1])


samples for  th__br__n.f_x.
thfebreen.fex.
thtebr.kn.fex.
tha.bro.n.fex.
th..bretn.fex.
theebrsin.fex.
theebrmon.fex.
theobrfkn.fex.
theibri.n.fex.
thoybrepn.fix.
theibr.kn.fex.
samples for  _u_st__n_.to_be._nsw_r__
fussthhnd.to.be.answar.i
sussthind.tombe.onswor.a
buesth.nd.to.be.vnswerts
ounsteond.tombe.inswir.a
buisthhng.tombe.answors.
sutsthhno.toube.wnsw.rbs
bursthhnd.to.be.answer.r
ouesthrnd.toube.answer.e
pupst.onk.tombe.wnswarsr
busstshne.toube.answer.r
samples for  i__at_._a_h_n_._e_r_i_g
inkato..athend.hear.ing
ittatf.pathend.tecr.ing
imnato..athung.beor.ing
irsate.hathens.ce.raing
iroati.sathane.he.rei.g
itnaty.saghind.fe.rcing
ielath.wathan..metr.ing
iotaty.oatheng.bear.ing
iosats.cathend.ceereing
inrath.eachend.ve.roing
samples for  q___t.___z._____t.__.___.__.
quuut.cphz.gdsawt.hp.apl.wr.
quuut.amhz.jbda.t.ew.oar..s.
quuut.oifz.iaritt.hp..fs.am.
quuut.hmsz.aa.wct.fa.sma..b.
quuut.bn.z.pd.ect.ss.c.h.tm.
quuut.qwaz.patlst.ko.iti.is.
quuut.datz.atah.t.th.ey..uo.
quuut.hocz.f

### Part 3
Describe a method for filling in the gaps by estimating the most likely letter for each position. Hint: you need to compute
$$
x_{-\alpha}^* = \arg\max_{x_{-\alpha}} p(x_{-\alpha}|x_{\alpha})
$$
Implement the method and print the results for the following test strings along with the log-probability  $\log p(x_{-\alpha}^*,x_{\alpha})$.
My algorithm is like:

Compute the probability for the first unknown element

Take the maximum element of the probability distribution (one dimensional). 

Assign the found element as first element. Decrease the number of multiplication by one. N--

Continue until end.


In [13]:
for i in range(0,len(test_strings)):
    print(problem_identify(test_strings[i],underscore_finder(test_strings[i]),T,True))

Log prob: -3.07433486577
the.br.an.fex.
Log prob: -8.32284514738
oursthend.to.be.answere.
Log prob: -11.636089996
in.ath.wathend.he.r.ing
Log prob: -22.9236424228
qur.t.thiz.the.at.an.the.an.


### Part 4
Argmax is a good way to predict most likely sequence of strings. However, I think it is too much constrained, because we dont use other probabilities. If we calculate the density function of probabilities we can have a better understanding of data. 

For further analysis.We can use factor graph message passing algorithm. That way it will provide better results to this problem. The algorithm will be less complicated. Since it supports more complicated forms of graphs. Thanks to that flexibility we can connect 

